# F1 Fantasy Program

## ReadMe

The goal of this program is to webscrape, and gather, F1 data. The data would then be saved, transformed, and analysed.

## Setup

In [130]:
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import glob

import fastf1

### Enable Caching

In [131]:
fastf1.Cache.enable_cache('cache')

### Ignore Warnings

In [132]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

## Data Aquisition

### Get the completed events of the season (year)

In [133]:
def get_season_completed_races(season, b_print=False):
    schedule = fastf1.get_event_schedule(season)
    completed_events = []
    for event in schedule['RoundNumber'][1:]:
        if schedule['EventDate'][i_race] < datetime.now():
            completed_events.append(schedule.iloc[[event]])
    if b_print:
        print(completed_events[1]['OfficialEventName'])
        
    return completed_events

### Load the data of the completed sessions

In [138]:
def get_session_data(season, event, session_type):
    session = fastf1.get_session(season, event, session_type)
    session.load()
    return session

In [141]:
def get_completed_sessions_data(season, event, session_type):
    completed_sessions = []
    for event in completed_events:
        session = get_session_data(season, event, session_type)
        completed_sessions.append(session)
    return completed_sessions

## TEST

In [139]:
season = 2023
session_num = 1
session_type = 'R'

session_data = get_session_data(season, session_num, session_type)
print(session_data)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v2.3.1]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
